In [ ]:
# Standard library imports
import os
import gc
import pickle
import re
import numpy as np
import pandas as pd

from hiclass import LocalClassifierPerParentNode
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, f1_score, make_scorer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, FeatureUnion

from tqdm import tqdm
from tqdm.notebook import tqdm as notebook_tqdm

# Activate pandas progress apply for tqdm
tqdm.pandas()

In [ ]:
def read_pickle(pickle_path):
    gc.disable()
    file = open(pickle_path, "rb")
    dataframe = pickle.load(file)
    gc.enable()
    file.close()
    return dataframe

In [ ]:
path = '/home/dsuser/Bugra/COCL/Data_Cleaning/CallCenter/cleaned_data_noname.pkl'

In [ ]:
data_raw = read_pickle(path)

In [ ]:
data_raw.head()

,channel,statement_id,grup,kategori,konu,detay,statement,answer,src_brm,statement_len,state_len
0,Call Center,2968459,Şikayet,Kart işlemleri,Nakit Çekim İşlemleri,Farklı banka ATM'den para alamadığı halde para...,musterimiz nolu kartindan akbank atm bolu atmd...,Debit Card nakit çekim itirazı işleme alınmışt...,Harcama İtirazları Birimi İnceleme Havuzu,25,25
1,Call Center,2969208,Şikayet,Kart işlemleri,Nakit Çekim İşlemleri,Farklı banka ATM'den para alamadığı halde para...,tarihinde numarali kartinizla tutarindaki atm ...,Debit Card nakit çekim itirazı işleme alınmışt...,Harcama İtirazları Birimi İnceleme Havuzu,55,55
2,Call Center,2965151,Şikayet,Kart işlemleri,Nakit Çekim İşlemleri,Başka Banka ATM'den TEB hesabı/Kredi Kartına y...,musterimiz nolu atm nolu kartiyla nolu hesabin...,Debit/Kredi Card nakit para yatırma itirazı iş...,Harcama İtirazları Birimi İnceleme Havuzu,34,34
3,Call Center,2969793,Şikayet,Kart işlemleri,Nakit Çekim İşlemleri,Farklı banka ATM'den para alamadığı halde para...,tarihinde numarali kartinizla tutarindaki atm ...,Nakit çekim itirazı işleme alınmıştır. ATM Ba...,Harcama İtirazları Birimi İnceleme Havuzu,37,37
4,Call Center,2965946,Şikayet,Kart işlemleri,Nakit Çekim İşlemleri,Başka Banka ATM'den TEB hesabı/Kredi Kartına y...,musterimiz garanti banka atm nolu karti para y...,Debit/Kredi Card nakit para yatırma itirazı iş...,Harcama İtirazları Birimi İnceleme Havuzu,32,32


In [ ]:
from sklearn import preprocessing
label_encoder_grp = preprocessing.LabelEncoder()
data_raw['grup_lbl'] = label_encoder_grp.fit_transform(data_raw['grup'])
label_encoder_ktg = preprocessing.LabelEncoder()
data_raw['kategori_lbl'] = label_encoder_ktg.fit_transform(data_raw['kategori'])
label_encoder_kn = preprocessing.LabelEncoder()
data_raw['konu_lbl'] = label_encoder_kn.fit_transform(data_raw['konu'])
label_encoder_detay = preprocessing.LabelEncoder()
data_raw['detay_lbl'] = label_encoder_detay.fit_transform(data_raw['detay'])

In [ ]:
data_raw.head()

,channel,statement_id,grup,kategori,konu,detay,statement,answer,src_brm,statement_len,state_len,grup_lbl,kategori_lbl,konu_lbl,detay_lbl
0,Call Center,2968459,Şikayet,Kart işlemleri,Nakit Çekim İşlemleri,Farklı banka ATM'den para alamadığı halde para...,musterimiz nolu kartindan akbank atm bolu atmd...,Debit Card nakit çekim itirazı işleme alınmışt...,Harcama İtirazları Birimi İnceleme Havuzu,25,25,5,9,96,177
1,Call Center,2969208,Şikayet,Kart işlemleri,Nakit Çekim İşlemleri,Farklı banka ATM'den para alamadığı halde para...,tarihinde numarali kartinizla tutarindaki atm ...,Debit Card nakit çekim itirazı işleme alınmışt...,Harcama İtirazları Birimi İnceleme Havuzu,55,55,5,9,96,177
2,Call Center,2965151,Şikayet,Kart işlemleri,Nakit Çekim İşlemleri,Başka Banka ATM'den TEB hesabı/Kredi Kartına y...,musterimiz nolu atm nolu kartiyla nolu hesabin...,Debit/Kredi Card nakit para yatırma itirazı iş...,Harcama İtirazları Birimi İnceleme Havuzu,34,34,5,9,96,56
3,Call Center,2969793,Şikayet,Kart işlemleri,Nakit Çekim İşlemleri,Farklı banka ATM'den para alamadığı halde para...,tarihinde numarali kartinizla tutarindaki atm ...,Nakit çekim itirazı işleme alınmıştır. ATM Ba...,Harcama İtirazları Birimi İnceleme Havuzu,37,37,5,9,96,177
4,Call Center,2965946,Şikayet,Kart işlemleri,Nakit Çekim İşlemleri,Başka Banka ATM'den TEB hesabı/Kredi Kartına y...,musterimiz garanti banka atm nolu karti para y...,Debit/Kredi Card nakit para yatırma itirazı iş...,Harcama İtirazları Birimi İnceleme Havuzu,32,32,5,9,96,56


In [ ]:
#with open('hiclass_grp_lb.pkl', 'wb') as f:
#    pickle.dump(label_encoder_grp, f)

#with open('hiclass_kategori_lbl.pkl', 'wb') as f:
#    pickle.dump(label_encoder_ktg, f)

#with open('hiclass_konu_lbl.pkl', 'wb') as f:
#    pickle.dump(label_encoder_kn, f)

#with open('hiclass_detay_lbl.pkl', 'wb') as f:
#    pickle.dump(label_encoder_detay, f)


In [ ]:
data_raw['label'] = data_raw.apply(lambda x: [x.grup_lbl, x.kategori_lbl, x.konu_lbl, x.detay_lbl], axis=1)

In [ ]:
train_list =[]
test_list = []
full_list = []

data_raw_grouped = data_raw.groupby(['grup','kategori','konu','detay']).apply(lambda x: x.sample(frac=1,random_state=24))
m = data_raw_grouped['label'].value_counts()
index = m[m.values>10].index
pd_temp = data_raw[data_raw['label'].isin(index)].copy()
labels=pd_temp.label.tolist()
pd_train, pd_test = train_test_split(pd_temp,
                                  test_size=0.3,
                                  random_state=101,
                                  stratify=labels)

In [ ]:
len(data_raw[data_raw['label'].isin(index)])

In [ ]:
len(pd_temp)

In [ ]:
len(pd_train)

In [ ]:
len(pd_test)

In [ ]:
pd_train.columns

In [ ]:
full_texts = pd_temp["statement"].tolist()
full_labels =  pd_temp["label"].tolist()
train_texts = pd_train["statement"].tolist()
train_labels =  pd_train["label"].tolist()
test_texts = pd_test["statement"].tolist()
test_labels =  pd_test["label"].tolist()

In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion
def feature_extractor():
    context_features = FeatureUnion(
        transformer_list=[
#             ('FastText', FtVectorizer()),
#            ('BERT', BERTVectorizer(folder=bert_folder) )
#            ('spacy', SpacyVectorizer(ner=True))
            ('word', TfidfVectorizer(
                strip_accents=None,
                analyzer='word',
                ngram_range=(1, 5),
                max_df=0.4,
                min_df=5,
                binary=False,
                use_idf=True,
                smooth_idf=True,
                sublinear_tf=True,
                max_features=500
            )),
            ('char', TfidfVectorizer(
                strip_accents=None,
                lowercase=False,
                analyzer='char',
                ngram_range=(2, 3),
                max_df=0.4,
                min_df=5,
                binary=False,
                use_idf=True,
                smooth_idf=True,
                sublinear_tf=10,
                max_features=700
            )),
        ]
    )

    features = FeatureUnion(
        transformer_list=[
            ('context', Pipeline(
                steps=[('vect', context_features)]
            )),
        ]
    )

    return features



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
import xgboost as xgb

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from hiclass import LocalClassifierPerNode
from bert_sklearn import BertClassifier
from hiclass import LocalClassifierPerParentNode
from sklearn.pipeline import Pipeline
lg = LogisticRegression(solver='liblinear')

pipeline = Pipeline(
    [
        ("extractor", feature_extractor()),
#        ('dim_reduction', reducer),
        ("lcppn", LocalClassifierPerParentNode(local_classifier=lg)),
    ],
    verbose=True
)

pipeline.fit(train_texts, train_labels)

In [ ]:
predictions = pipeline.predict(test_texts)

In [ ]:
pred = [[int(p[0]), int(p[1]), int(p[2]) , int(p[3])] for p in predictions]

In [ ]:
def met(pred, y_test):
    true_score = 0
    for i in range(0,len(pred)):
        if pred[i][0] == y_test[i][0] and pred[i][1] == y_test[i][1] and pred[i][2] == y_test[i][2] and pred[i][3] == y_test[i][3]:
            true_score +=1
        elif pred[i][0] == y_test[i][0] and pred[i][1] == y_test[i][1] and pred[i][2] == y_test[i][2] and pred[i][3] != y_test[i][3]:
            true_score += (1/4)*3
        elif pred[i][0] == y_test[i][0] and pred[i][1] == y_test[i][1] and pred[i][2] != y_test[i][2] and pred[i][3] != y_test[i][3]:
            true_score += (1/4)*2
        elif pred[i][0] == y_test[i][0] and pred[i][1] != y_test[i][1] and pred[i][2] != y_test[i][2] and pred[i][3] != y_test[i][3]:
            true_score += (1/4)
    return true_score/len(pred)

In [ ]:
from hiclass import metrics
metrics.f1(pred, test_labels)

In [ ]:
print("Score : "+str(met(pred,test_labels)))

In [ ]:
with open("modal_v2_noname_cleaned.pkl","wb") as file:
    pickle.dump(pipeline,file)

In [ ]:
with open("modal_v2_noname_cleaned.pkl","rb") as file:
    pipeline = pickle.load(file)

In [ ]:
predictions = pipeline.predict(test_texts)

In [ ]:
pred = [[int(p[0]), int(p[1]), int(p[2]) , int(p[3])] for p in predictions]

In [ ]:
print("Score : "+str(met(pred,test_labels)))

Score : 0.8326317664776718


In [ ]:
from hiclass import metrics
metrics.f1(pred, test_labels)

0.8489844380491638